# Implementering av et dyreekspertsystem

Et eksempel fra [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

I dette eksempelet skal vi implementere et enkelt kunnskapsbasert system for å identifisere et dyr basert på noen fysiske egenskaper. Systemet kan representeres av følgende AND-OR-tre (dette er en del av hele treet, vi kan enkelt legge til flere regler):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.no.png)


## Vår egen ekspertsystemskall med bakoverinferens

La oss prøve å definere et enkelt språk for kunnskapsrepresentasjon basert på produksjonsregler. Vi vil bruke Python-klasser som nøkkelord for å definere regler. Det vil i hovedsak være tre typer klasser:
* `Ask` representerer et spørsmål som må stilles til brukeren. Den inneholder settet med mulige svar.
* `If` representerer en regel, og det er bare syntaktisk sukker for å lagre innholdet i regelen.
* `AND`/`OR` er klasser som representerer AND/OR-grener i treet. De lagrer bare listen over argumenter inni. For å forenkle koden er all funksjonalitet definert i foreldresklassen `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

I vårt system vil arbeidsminnet inneholde listen over **fakta** som **attributt-verdi-par**. Kunnskapsbasen kan defineres som en stor ordbok som kobler handlinger (nye fakta som skal settes inn i arbeidsminnet) til betingelser, uttrykt som OG-ELLER-uttrykk. I tillegg kan noen fakta bli `Spurt`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

For å utføre baklengs slutning, vil vi definere klassen `Knowledgebase`. Den vil inneholde:
* Arbeidsminne (`memory`) - en ordbok som kobler attributter til verdier
* Kunnskapsbase (`rules`) i formatet som definert ovenfor

To hovedmetoder er:
* `get` for å hente verdien av et attributt, og utføre slutning hvis nødvendig. For eksempel, `get('color')` vil hente verdien av en fargeplass (den vil spørre hvis nødvendig, og lagre verdien for senere bruk i arbeidsminnet). Hvis vi spør `get('color:blue')`, vil den spørre om en farge, og deretter returnere `y`/`n` verdi avhengig av fargen.
* `eval` utfører den faktiske slutningen, dvs. traverserer AND/OR-treet, evaluerer delmål, osv.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Nå la oss definere vår dyrekunnskapsbase og utføre konsultasjonen. Merk at denne forespørselen vil stille deg spørsmål. Du kan svare ved å skrive `y`/`n` for ja-nei-spørsmål, eller ved å spesifisere et tall (0..N) for spørsmål med lengre svaralternativer.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Bruke PyKnow for Fremoverrettet Infernens

I det neste eksempelet skal vi prøve å implementere fremoverrettet inferens ved hjelp av et av bibliotekene for kunnskapsrepresentasjon, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** er et bibliotek for å lage fremoverrettede inferenssystemer i Python, som er designet for å ligne på det klassiske gamle systemet [CLIPS](http://www.clipsrules.net/index.html).

Vi kunne også ha implementert fremoverrettet kjeding selv uten store problemer, men naive implementeringer er vanligvis ikke særlig effektive. For mer effektiv regelmatching brukes en spesiell algoritme, [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Vi vil definere systemet vårt som en klasse som underklasser `KnowledgeEngine`. Hver regel er definert av en separat funksjon med `@Rule`-annotasjon, som spesifiserer når regelen skal utløses. Inne i regelen kan vi legge til nye fakta ved å bruke `declare`-funksjonen, og å legge til disse faktaene vil føre til at flere regler blir kalt av fremover-slutningsmotoren.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Når vi har definert en kunnskapsbase, fyller vi arbeidsminnet vårt med noen innledende fakta, og deretter kaller vi `run()`-metoden for å utføre slutningen. Du kan se som et resultat at nye utledede fakta legges til arbeidsminnet, inkludert det endelige faktumet om dyret (hvis vi har satt opp alle de innledende faktaene riktig).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
